NOT WORKNG WITH BINDER YET!

**import libraries**

In [1]:
# notebook or inline or qt
%matplotlib qt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import signal
from matplotlib import animation

# from IPython.display import HTML

---
**generate signals**

In [2]:
sr = 100 # sample rate

t = np.arange(-5, 5, 1 / sr) # has to be symmetric around zero
f = np.fft.fftshift(np.fft.fftfreq(t.size, 1 / sr))

# generate first signal (rect)
data1 = np.zeros(t.size)
data1[(t>=0) & (t<=1)] = 1

# generate second signal
# # decaying exponential
# alpha = 3
# data2 = np.exp(-alpha * t)
# data2[t<0]=0

T = 1/1
T0 = 0.
# rect
data2 = np.zeros(t.size)
data2[(t>=(0+T0)) & (t<=(T+T0))] = 1.0

# plot both signals (figure 1)
fig, ax = plt.subplots(1,2, figsize=(2*4, 4), num=1)
ax[0].cla()
ax[0].plot(t, data1)
ax[0].set(ylabel='amplitude / a.u.', xlabel='time / s')
ax[0].grid(True) 

ax[1].cla()
ax[1].plot(t, data2)
ax[1].set(ylabel='amplitude / a.u.', xlabel='time / s')
# ax[1].set_ylim((0, 1))
ax[1].grid(True) 

plt.show()

---
**plot swapped and enlarged signals**

In [3]:
# swap second signal (flip around t=0)
data2 = data2[::-1]

# plot both signals (figure 2)
fig, ax = plt.subplots(1,2, figsize=(2*4, 4), num=2)
ax[0].cla()
ax[0].plot(t, data1)
ax[0].set(ylabel='amplitude / a.u.', xlabel='time / s')
ax[0].grid(True) 

ax[1].cla()
l, = ax[1].plot(t, data2)
ax[1].set(ylabel='amplitude / a.u.', xlabel='time / s')
l.set_color([1, 0.5, 0])
ax[1].grid(True) 

plt.show()

---
**visualize convolution operation**

$y(\tau)=a(t)\ast b(t)=\int_{-\infty}^{\infty}a(t)\cdot b(\tau -t)dt$

In [4]:
# generate shift vector tau
dtau = 1./(sr)
tauMinMax = [-2, 2]; #[-0.5, 3]
tau = np.arange(tauMinMax[0], tauMinMax[1], dtau)

# prepare plots
fig, ax = plt.subplots(1,3, figsize=(3*5, 4), num=3)

line00, line01 = ax[0].plot([], [], [], [])
line01.set_color([1, 0.5, 0])
poly0 = ax[0].fill_between([], [], animated=True, color=[0, 0.8, 0.5], alpha=0.7)
ax[0].set(ylabel='amplitude / a.u.', xlabel='time / s')
ax[0].set_xlim(np.min(t), np.max(t))
ax[0].set_ylim(0, 1.1)  
ax[0].grid(True)  

line10, = ax[1].plot([], [])
poly1 = ax[1].fill_between([], [], animated=True, color=[0, 0.8, 0.5], alpha=0.7)
ax[1].set(ylabel='amplitude / a.u.', xlabel='time / s')
ax[1].set_xlim(np.min(t), np.max(t))
ax[1].set_ylim(0, 1.1) 
line10.set_color([0, 0, 0])
ax[1].grid(True) 

line20, = ax[2].plot([], [])
line21, = ax[2].plot([], [])
line20.set_color([0, 0.8, 0.5])
line21.set_color([1, 0, 0])
line21.set_marker('o')
ax[2].set(ylabel='amplitude / a.u.', xlabel='shift  / s')
ax[2].set_xlim(np.min(tau), np.max(tau))
# ax[2].set_ylim(0, (np.exp(-alpha*1)+1)/alpha * 1.1) 
ax[2].set_ylim(0, 1*T*1.1) 
ax[2].grid(True) 

plt.subplots_adjust(wspace=0.5)

# function to animate the convolution 
integral = np.zeros_like(tau)
def update(i):  
            
    line00.set_data(t, data1)
    line01.set_data(t, np.roll(data2, int(tau[i]*sr)))
        
    fOverlap = data1 * np.roll(data2, int(tau[i]*sr))
    poly0.set_verts((np.c_[t, fOverlap],))
    poly1.set_verts((np.c_[t, fOverlap],))
    line10.set_data(t, fOverlap)  
    
#     section = np.arange(, 1, 1/20.)
#     plt.fill_between(section,f(section))

    integral[i] = sum(data1 * np.roll(data2, int(tau[i]*sr))) / sr
    line20.set_data(tau, integral)
    line21.set_data(tau[i], integral[i])
    
    return poly0, poly1, line00, line01, line10, line20, line21

ani = animation.FuncAnimation(fig, update, frames=tau.size, interval=20, blit=True)
# ani.save('convolutionDemoExp.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

# HTML(ani.to_jshtml())
# HTML(ani.to_html5_video())
plt.show()